# Autoimmune colocalisations

The purpose of this notebook is to extract colocalisations for GWAS credible set associated with autoimmune diseases, including additional metadata about the studies.

1. **Download datasets from Open Targets Platform**

In [14]:
%%bash
release="25.06"
datasets=("study" "credible_set" "colocalisation_coloc" "colocalisation_ecaviar" "target" "disease" "biosample")
for dataset in "${datasets[@]}"
    do mkdir -p ../tmp/"${dataset}"
    # Rsync the data from EBI FTP server
    rsync -rpltvz --delete rsync.ebi.ac.uk::pub/databases/opentargets/platform/${release}/output/${dataset} ../tmp/
done

2. **Python environment and Spark session**

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Starting a Spark session
spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

25/10/14 17:14:46 WARN Utils: Your hostname, MW4X0GG4FP resolves to a loopback address: 127.0.0.1; using 172.23.49.88 instead (on interface en0)
25/10/14 17:14:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/14 17:14:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3. **Read downloaded datasets**

In [2]:
# Reading datasets
credible_set = spark.read.parquet("../tmp/credible_set")

# Union two colocalisation datasets "colocalisation_ecaviar" and "colocalisation_coloc"
colocalisation_coloc = spark.read.parquet("../tmp/colocalisation_coloc")
colocalisation_ecaviar = spark.read.parquet("../tmp/colocalisation_ecaviar")
colocalisation = colocalisation_coloc.unionByName(
    colocalisation_ecaviar, allowMissingColumns=True
)

# Read other datasets
study = spark.read.parquet("../tmp/study")
target = spark.read.parquet("../tmp/target")
disease = spark.read.parquet("../tmp/disease")
biosample = spark.read.parquet("../tmp/biosample")

4. **Finding all autoimmune diseases according to EFO**

In [3]:
autoimmune_efo = "EFO_0005140"
autoimmune_diseases = (
    disease.filter(f.col("id") == autoimmune_efo)
    .select(f.explode("descendants").alias("diseaseId"))
    .join(
        disease.select(f.col("id").alias("diseaseId"), "name"),
        on="diseaseId",
        how="left",
    )
)
autoimmune_diseases.show(truncate=False)

+-------------+--------------------------------------------------------------------------------+
|diseaseId    |name                                                                            |
+-------------+--------------------------------------------------------------------------------+
|MONDO_0012500|chilblain lupus 1                                                               |
|MONDO_0010894|maturity-onset diabetes of the young type 3                                     |
|MONDO_0024278|proctocolitis                                                                   |
|EFO_0005626  |pancolitis                                                                      |
|EFO_0008613  |pemphigus vegetans                                                              |
|EFO_0803379  |anti-GAD65 autoimmune neurological syndromes                                    |
|MONDO_0005301|multiple sclerosis                                                              |
|EFO_0008605  |IgG/IgA pemphig

5. **Finding all GWAS studies for autoimmune diseases**


In [4]:
auto_gwas_studies = study.withColumn("diseaseId", f.explode("diseaseIds")).join(
    autoimmune_diseases, on="diseaseId", how="inner"
)
auto_gwas_studies.show(1, vertical=True, truncate=False)

25/10/14 17:14:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------
 diseaseId                          | EFO_0002689                                                                                                          
 studyId                            | GCST004227                                                                                                           
 geneId                             | NULL                                                                                                                 
 projectId                          | GCST                                                                                                                 
 studyType                          | gwas                                                                                                                 
 traitFromSource                    | Obstetric antiphospholipid

6. **Extracting all credible sets in autoimmune studies**


In [5]:
auto_cs = auto_gwas_studies.join(credible_set, on="studyId", how="inner")
auto_cs.count()

8524

7. **Find colocalising molecular QTLs for each credible set**

The colocalisation dataset contains all GWAS - GWAS and GWAS - molQTL credible sets. This order is persisted, therefore all molQTL credible sets are in the right side of the colocalisation results. 

In [6]:
auto_cs_colocalisations = (
    auto_cs.withColumnRenamed("studyLocusId", "leftStudyLocusId")
    .alias("gwas")
    # Bring study information for the GWAS study
    .join(
        study.alias("gwas_study"),
        on=[f.col("gwas_study.studyId") == f.col("gwas.studyId")],
        how="inner",
    )
    # Bring colocalisation results
    .join(
        colocalisation.alias("colocalisation").filter(
            # Sensible filter for colocalisation results
            (f.col("clpp") > 0.01) | (f.col("h4") > 0.8)
        ),
        on="leftStudyLocusId",
        how="inner",
    )
    # ignore GWAS - GWAS colocalisations
    .filter(f.col("rightStudyType") != "gwas")
    # Bring molQTL credible set information
    .join(
        credible_set.alias("molQTL").withColumnRenamed(
            "studyLocusId", "rightStudyLocusId"
        ),
        on="rightStudyLocusId",
        how="inner",
    )
    # Bring study information for the molQTL study
    .join(
        study.alias("molQTL_study"),
        on=[f.col("molQTL_study.studyId") == f.col("molQTL.studyId")],
        how="inner",
    )
    # Add approved symbol for the molQTL gene
    .join(
        target.select("approvedSymbol", "id").alias("molQTL_target"),
        on=f.col("molQTL_target.id") == f.col("molQTL_study.geneId"),
        how="left",
    )
    # Add biosample information
    .join(
        biosample.select("biosampleId", "biosampleName").alias("molQTL_biosample"),
        on=f.col("molQTL_study.biosampleId") == f.col("molQTL_biosample.biosampleId"),
        how="left",
    )
)
# Count of moQTL colocalising with autoimmune disease credible sets across tissues/cell types
auto_cs_colocalisations.count()

207489

8. **Select columns of interest to print/write**

In [ ]:
cs_out = auto_cs_colocalisations.select(
    # About the study
    "gwas_study.studyId",
    "gwas_study.publicationJournal",
    "gwas_study.publicationDate",
    "gwas_study.nSamples",
    "gwas_study.ldPopulationStructure",
    "gwas_study.traitFromSource",
    "gwas_study.diseaseIds",
    f.col("name").alias("diseaseName"),
    # About the credible set
    "gwas.leftStudyLocusId",
    "gwas.pValueMantissa",
    "gwas.pValueExponent",
    "gwas.beta",
    "gwas.standardError",
    "gwas.fineMappingMethod",
    "gwas.hasSumstats",
    f.col("gwas.variantId").alias("leadVariant"),
    # About the molecular QTL
    "molQTL_target.approvedSymbol",
    "molQTL_target.id",
    "molQTL_study.biosampleId",
    "molQTL_biosample.biosampleName",
    "molQTL_study.condition",
    "molQTL_study.projectId",
    "molQTL.beta",
    "molQTL.standardError",
    "molQTL.pValueMantissa",
    "molQTL.pValueExponent",    
)

cs_out.show(truncate=False)

# This dataframe can be written to different formats including parquet file:
# cs_out.write.parquet("../tmp/autoimmune_credible_set_parquet", mode="overwrite")
# or csv:
# cs_out.drop("ldPopulationStructure").coalesce(1).write.csv(
#     "../tmp/autoimmune_credible_set_csv",
#     mode="overwrite",
#     header=True,
# )

+------------+------------------+---------------+--------+-------------------------------------------------------+---------------------------------+-------------+--------------------------+--------------------------------+--------------+--------------+--------------------+-------------+-----------------+-----------+---------------+--------------+---------------+--------------+-------------+---------+-----------+--------------------+-------------+--------------+--------------+
|studyId     |publicationJournal|publicationDate|nSamples|ldPopulationStructure                                  |traitFromSource                  |diseaseIds   |diseaseName               |leftStudyLocusId                |pValueMantissa|pValueExponent|beta                |standardError|fineMappingMethod|hasSumstats|leadVariant    |approvedSymbol|id             |biosampleId   |biosampleName|condition|projectId  |beta                |standardError|pValueMantissa|pValueExponent|
+------------+------------------+-----